**Note to grader:** Each question is assigned with a score. The final score will be (sum of actual scores)/(sum of maximum scores)*100. The grading rubrics are shown in the TA guidelines.

In [2]:
# Grader's area
import numpy as np

maxScore = 0


# **Assignment 5**
<br>

<font>

In [3]:
from IPython.display import Math
from IPython.display import Latex

import numpy as np
import os
import struct

from sklearn.metrics import classification_report,confusion_matrix
from sklearn.neural_network import MLPClassifier

----------------
----------------
----------------

## <font color='blue'> Question 0. MLP with scikit-learn: Explore outputs for the MNIST dataset [40 pts]</font>

The purpose of this question is to get some practice time with MLP classifiers. We will be using the the MNIST data set.




In [4]:
# Let's first load the MNIST data set.

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split


X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
y = y.astype(int)
X = ((X / 255.) - .5) * 2
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=10000, random_state=123, stratify=y)


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


<br> **Q0-0**: This is the classifier we built in class. Find a setting for *batch_size* and *max_iter*, so that the subsequent fit function **converges**. Converge here is defined as: the training loss does not improve by more than tol=1e-4 for n_iter_no_change=10 consecutive passes over the training set. [10 pts]





In [5]:

mlp_mnist = MLPClassifier(verbose = True, hidden_layer_sizes=(50,), batch_size = 150, \
                          max_iter=1000, solver='sgd', activation='logistic',\
                          learning_rate = 'constant', learning_rate_init = 0.001, random_state=1)



In [6]:
# Fit the model
mlp_mnist.fit(X_train, y_train)

# Get the loss curve
loss_curve = mlp_mnist.loss_curve_

# Initialize variables
n_iter_no_change = 10
tol = 1e-4
no_improvement_count = 0

# Check for convergence
for i in range(1, len(loss_curve)):
    # Calculate the difference between current loss and previous loss
    loss_diff = loss_curve[i-1] - loss_curve[i]

    # If the difference is less than the tolerance
    if loss_diff < tol:
        no_improvement_count += 1
    else:
        no_improvement_count = 0

    # If no improvement for n_iter_no_change consecutive iterations, then it has converged
    if no_improvement_count >= n_iter_no_change:
        print(f"The model has converged at iteration {i}")
        break

Iteration 1, loss = 2.03579884
Iteration 2, loss = 1.52119642
Iteration 3, loss = 1.15936386
Iteration 4, loss = 0.93362604
Iteration 5, loss = 0.78873279
Iteration 6, loss = 0.69038101
Iteration 7, loss = 0.62036665
Iteration 8, loss = 0.56823691
Iteration 9, loss = 0.52819096
Iteration 10, loss = 0.49650924
Iteration 11, loss = 0.47093555
Iteration 12, loss = 0.44962928
Iteration 13, loss = 0.43191411
Iteration 14, loss = 0.41679350
Iteration 15, loss = 0.40363286
Iteration 16, loss = 0.39232438
Iteration 17, loss = 0.38209999
Iteration 18, loss = 0.37309803
Iteration 19, loss = 0.36505994
Iteration 20, loss = 0.35772369
Iteration 21, loss = 0.35097184
Iteration 22, loss = 0.34489513
Iteration 23, loss = 0.33923360
Iteration 24, loss = 0.33405509
Iteration 25, loss = 0.32898299
Iteration 26, loss = 0.32452258
Iteration 27, loss = 0.32011973
Iteration 28, loss = 0.31615649
Iteration 29, loss = 0.31222310
Iteration 30, loss = 0.30861018
Iteration 31, loss = 0.30511421
Iteration 32, los

In [7]:
predictions = mlp_mnist.predict(X_test)
probability_predictions = mlp_mnist.predict_proba(X_test)

**Q0-1**: Write a short function that outputs the **second best** prediction for each point, i.e. the label which gets the second highest probability in the softmax output. [10 pts]

In [8]:
def second_best_predictions(proba_predictions):
    # Use argsort to get sorted indices and then take second last element along axis 1
    second_best_pred_indices = np.argsort(proba_predictions, axis=1)[:,-2]
    return second_best_pred_indices

# Get the second best predictions
second_best_preds = second_best_predictions(probability_predictions)
print(second_best_preds)

[5 4 0 ... 3 2 3]


**Q0-2**: For each data point misclassified by the model, find if the second best prediction is actually the correct label. Calculate the percentage of missclassified points, that are correctly classified based on the second best prediction. [10 pts]

In [9]:
def calculate_percentage_correct_second_best(predictions, second_best_preds, y_test):
    # Find the indices of the misclassified points
    misclassified_indices = predictions != y_test

    # Get the second best predictions and the true labels for the misclassified points
    second_best_preds_misclassified = second_best_preds[misclassified_indices]
    y_test_misclassified = y_test[misclassified_indices]

    # Find where the second best prediction is the correct label
    correct_second_best = second_best_preds_misclassified == y_test_misclassified

    # Calculate the percentage of misclassified points that are correctly classified by the second best prediction
    percentage_correct_second_best = correct_second_best.mean() * 100

    return percentage_correct_second_best

# Calculate the percentage
percentage = calculate_percentage_correct_second_best(predictions, second_best_preds, y_test)
print(f"The percentage of misclassified points that are correctly classified based on the second best prediction is {percentage}%.")
# give comments on findings
#This means that The finding that 66% of misclassified points are correctly classified based on the second best prediction suggests that the classifier is often “close” to the correct answer even when it makes a mistake.
#the correct label is in the top two most likely labels according to the classifier. This could indicate that these misclassified images may have features that make them particularly difficult to classify like these is messy handwriting

The percentage of misclassified points that are correctly classified based on the second best prediction is 66.56441717791411%.


**Q0-3**: From a human point of view, we can perhaps see a frequent similarity between handritten digits 3 and 5 (i.e. sometimes we have to pay closer attention to figure out what the true written digit is). We may want to ask if the MLP also detects digit similarities.  
<br>
 Consider then the subset $X_i$ of data points that have label $i$ (e.g. all data points with label 5). For each label $j\neq i$, report the frequency by which label $j$ (e.g. label 3) shows up as second best prediction for points in $X_i$. [10 pts]
<br>
<br>
***Further Clarification***: Suppose we have 100 data points with label i=5. Let's say that for 28 of these 100 points, the second predicted label is j=3. Then, the frequency of j=3 as second-best prediction to i=5 is 28%.  We want to compute that frequency for every i and j.

Example output:
> i = 0: 986 (9.86%)

  j = 1: 2 (0.203%)

  j = 2: 248 (25.152%)

  j = 3: 8 (0.811%)

  j = 4: 4 (0.406%)

  j = 5: 270 (27.383%)

  j = 6: 188 (19.067%)

  j = 7: 56 (5.68%)

  j = 8: 52 (5.274%)

  j = 9: 150 (15.213%)

<br>  
 Give your code and comment on your findings

In [20]:
### Your code here. Also give some comments about your findings
def calculate_second_best_frequencies(y_test, second_best_preds):
    # Initialize a dictionary to store the frequencies
    frequencies = {}

    # Loop over each unique label in y_test
    for i in np.unique(y_test):
        # Get the second best predictions for the current label
        second_best_preds_i = second_best_preds[y_test == i]

        # Calculate the frequency of each label in the second best predictions
        frequencies_i = np.bincount(second_best_preds_i) / len(second_best_preds_i)

        # Store the frequencies in the dictionary
        frequencies[i] = frequencies_i

    # Return the dictionary of frequencies
    return frequencies

# Calculate the frequencies
frequencies = calculate_second_best_frequencies(y_test, second_best_preds)

# Print the frequencies
for i, frequencies_i in frequencies.items():
    print(f"i = {i}:")
    for j, frequency in enumerate(frequencies_i):
        if j != i:
            print(f"  j = {j}: {frequency * 100:.3f}%")




#This can provide insights into which digits the model often confuses with each other.
#for instance when the true label is 0, the second-best prediction is most frequently 5 (43.205%). This suggests that the model often confuses 0s and 5s.
#for i=4 the highest frequently is 9 (58.359%). This suggests that the model often confuses 4s and 9s.
#This shows the models confusing

i = 0:
  j = 1: 0.000%
  j = 2: 19.675%
  j = 3: 1.014%
  j = 4: 0.203%
  j = 5: 43.205%
  j = 6: 11.765%
  j = 7: 4.057%
  j = 8: 4.564%
  j = 9: 15.010%
i = 1:
  j = 0: 0.000%
  j = 2: 8.978%
  j = 3: 20.444%
  j = 4: 0.178%
  j = 5: 1.689%
  j = 6: 2.844%
  j = 7: 30.844%
  j = 8: 32.178%
  j = 9: 2.044%
i = 2:
  j = 0: 3.604%
  j = 1: 7.307%
  j = 3: 49.550%
  j = 4: 4.204%
  j = 5: 0.601%
  j = 6: 8.609%
  j = 7: 7.107%
  j = 8: 16.316%
  j = 9: 0.400%
i = 3:
  j = 0: 1.176%
  j = 1: 8.235%
  j = 2: 12.549%
  j = 4: 0.392%
  j = 5: 27.647%
  j = 6: 0.294%
  j = 7: 6.275%
  j = 8: 14.314%
  j = 9: 25.784%
i = 4:
  j = 0: 0.615%
  j = 1: 0.000%
  j = 2: 6.769%
  j = 3: 0.308%
  j = 5: 6.359%
  j = 6: 9.128%
  j = 7: 14.359%
  j = 8: 1.949%
  j = 9: 58.359%
i = 5:
  j = 0: 7.317%
  j = 1: 4.656%
  j = 2: 1.552%
  j = 3: 33.814%
  j = 4: 3.880%
  j = 6: 6.984%
  j = 7: 1.441%
  j = 8: 27.827%
  j = 9: 8.980%
i = 6:
  j = 0: 7.536%
  j = 1: 1.731%
  j = 2: 50.000%
  j = 3: 0.102%
  j =

In [11]:
# For grader use only



maxScore = maxScore + 40


------------------------------
------------------------------
------------------------------

## <font color='blue'>Question 1: Working with warm start for focused training [10 pts by 4 = 40 pts]



**Q1-0**. MLP classifiers in scikit-learn have an option for so-called 'warm start'. Do a bit of research to find out what it is. Then:

Give an example of warm start. Specifically, in scenario 1:
train the network for $k$ epochs (max_iter=k), and then without using warm start
continue the training for another $k$ epochs; in scenario 2: train the network for $k$ epochs, and then using warm start
continue the training for another $k$ epochs. Check the outputs from training, you will find the different between two scenarios.
Here $k$ is a parameter of your choice, you can set $k=5$. [10 points]





In [21]:
### Your code here. Also give some comments about your findings
# NO WARM UP
# Train the network for k epochs
K=5
mlp_mnist = MLPClassifier(hidden_layer_sizes=(50,), batch_size=30, max_iter=K, random_state=1)
mlp_mnist.fit(X_train, y_train)

# Continue the training for another k epochs
mlp_mnist = MLPClassifier(hidden_layer_sizes=(50,), batch_size=30, max_iter=K, random_state=1)
mlp_mnist.fit(X_train, y_train)

# WARM UP
# Train the network for k epochs
mlp_mnist = MLPClassifier(hidden_layer_sizes=(50,), batch_size=30, max_iter=K, random_state=1, warm_start=True)
mlp_mnist.fit(X_train, y_train)

# Continue the training for another k epochs
mlp_mnist.max_iter += K
mlp_mnist.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(batch_size=30, hidden_layer_sizes=(50,), max_iter=10,
              random_state=1, warm_start=True)


The training set usally remains the same after a warm start. Here, we will use a very similar bit not identical dataset after the warm start:
<br>
<br>
**Q1-1**. Randomly divide the MNIST dataset into three part: train1 (50000), train2(10000), test (10000). Make sure that all number classes are presented in both train1, and train2. Hints: use train_test_split(). [10 points]


In [22]:
# Evaluate the models
accuracy_strategy1 = mlp_mnist.score(X_test, y_test)

print(f"Accuracy with Strategy 1: {accuracy_strategy1 * 100:.2f}%")


Accuracy with Strategy 1: 96.24%


In [23]:
### Your code here. Also give some comments about your findings
from sklearn.model_selection import train_test_split

# First, split the data into train (60000) and test (10000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10000, random_state=123, stratify=y)

# Then, split the train data into train1 (50000) and train2 (10000)
X_train1, X_train2, y_train1, y_train2 = train_test_split(X_train, y_train, test_size=10000, random_state=123, stratify=y_train)

print(f"Train1 size: {X_train1.shape[0]}")
print(f"Train2 size: {X_train2.shape[0]}")
print(f"Test size: {X_test.shape[0]}")




Train1 size: 50000
Train2 size: 10000
Test size: 10000


**Q1-2**. Use warm start and train your model using the train1 dataset for $2k$ epochs. Then, using warm start, continute the training with train2 dataset for another $2k$ epochs. [10 points]
<br>
<br>
Here $k$ is the same parameter you used above. Notice that each point
is considered $2k$ times in this training, exactly as in part 0.
<br>
<br>
In effect this process of training tries spend more time learning half of the training set, and then moves to the second part of the training set to spend equal time.


In [24]:
### Your code here. Also give some comments about your findings
# Train the network on train1 dataset for 2k epochs
mlp_mnist = MLPClassifier(hidden_layer_sizes=(50,), batch_size=30, max_iter=2*K, random_state=1, warm_start=True)
mlp_mnist.fit(X_train1, y_train1)

# Continue the training on train2 dataset for another 2k epochs
mlp_mnist.max_iter += 2*K
mlp_mnist.fit(X_train2, y_train2)





/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(batch_size=30, hidden_layer_sizes=(50,), max_iter=20,
              random_state=1, warm_start=True)

In [25]:
# Evaluate the models
accuracy_strategy2 = mlp_mnist.score(X_test, y_test)

print(f"Accuracy with Strategy 2: {accuracy_strategy2 * 100:.2f}%")


Accuracy with Strategy 2: 95.33%


**Q1-3**. Which of the two above strategies
works better? (Q1-0 or Q1-2). Feel free to experiment and report your findings.
 [10 pts]

 Strategy 1 is works better. Strategy 1 has an accuracy of 96.24% and stratergy 2 has a 95.33% so strategy 1 has a .48% increase in accuracy.

In [15]:
# For grader use only

maxScore = maxScore + 40


--------------------------
--------------------------
--------------------------

##  <font color='blue'> Q2. How Powerful are Neural Networks? [20 pts]

There is a lot of hype around Neural Networks, and their ability to solver problems. This is of course justified, but we should always be aware of their limitations.


Recall that every row in the matrix $X$  above comes from an image. Every entry of a row corresponds to a (grayscale) pixel. In particular, the first colum of $X$ contains the grayscale values of the first pixels of the images in the data set.
<br>
<br>
We will now 'hide' the label of the image, inside the image itself.
Here is one way to do that: Replace the top-left pixel of the image (which is a value from 0 to 255) with the label for that image. For example, if the image
contains number 3, then the first pixel of the image will now be set to 3. (In fact, let's set this to 3/255 because we already divided all pixel values by 255).

**Q2-0.** Create a new dataset $X1$, where each row is the same as the corresponding row of $X$, except in the first column, which contains the label of the corresponding image, divided by 255. Then split
the set $X1$ into a training set and a test set, as we did with $X$. [10 pts]

Hints: make a copy of X and work on the copy data: X1 = X.copy()




In [16]:
# Make a copy of X
X1 = X.copy()

# Replace the first column with the label of the corresponding image, divided by 255
X1.iloc[:, 0] = y / 255.

# Split the data into a training set and a test set
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y, test_size=10000, random_state=123, stratify=y)

print(f"X1_train size: {X1_train.shape[0]}")
print(f"X1_test size: {X1_test.shape[0]}")


X1_train size: 60000
X1_test size: 10000


**Q2-1.** With the label of the image 'planted' in the image itself, a Neural Network can achieve accuracy 100%. This is because all the weights connecting to pixels $2...28^2$ can be set to 0, and so effectively the only feature that will be taken into account is the first pixel. The question is whether an MLP can indeed 'discover' this weighting via training. [10 pts]
<br>
<br>
Use an MLP classifier  with one hidden layer of any size you want. Train it on $X1$ and report the accuracy. Does it reach 100\%? Is the accuracy better relative to training with the original input $X$?




In [17]:
# for grader use
from sklearn.metrics import accuracy_score

# Define the MLP classifier
mlp_mnist_X1 = MLPClassifier(hidden_layer_sizes=(50,), batch_size=30, max_iter=2*K, random_state=1)

# Train the classifier on the X1_train dataset
mlp_mnist_X1.fit(X1_train, y1_train)

# Make predictions on the X1_test dataset
predictions_X1 = mlp_mnist_X1.predict(X1_test)

# Calculate the accuracy
accuracy_X1 = accuracy_score(y1_test, predictions_X1)
print(f"Accuracy with X1: {accuracy_X1 * 100:.2f}%")


# Define the MLP classifier
mlp_mnist_X = MLPClassifier(hidden_layer_sizes=(50,), batch_size=30, max_iter=2*K, random_state=1)

# Train the classifier on the X_train dataset
mlp_mnist_X.fit(X_train, y_train)

# Make predictions on the X_test dataset
predictions_X = mlp_mnist_X.predict(X_test)

# Calculate the accuracy
accuracy_X = accuracy_score(y_test, predictions_X)
print(f"Accuracy with X: {accuracy_X * 100:.2f}%")


#The accuracy doesn't quite hit 100%, instead is 95.33%.
#This is a .43% decrease from the original X making X better than X1

# Total Grade Calculation

maxScore = maxScore + 20
#score = actualScore*100/maxScore

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy with X1: 95.33%
Accuracy with X: 95.76%


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
